In [7]:
import numpy as np;
import os;
import util;
import visualize;
import random;

def comparativeLossViz(img_dirs,file_post,loss_post,range_batches,range_images,out_file_html,dir_server,img_caption_pre=None):
    img_files_all=[];
    captions_all=[];
    if img_caption_pre is not None:
        assert len(img_caption_pre)==len(img_dirs);
    batch_str=True;
    if range_batches is None:
        range_batches=range(1);
        batch_str=False;
    
    for batch_num in range_batches:
    # range(1,num_batches+1):
        for im_num in range_images:
            
            for idx_img_dir,img_dir in enumerate(img_dirs):
                files_curr_list=[];
                captions_curr_list=[];

                if loss_post is None:
                    loss_str='';
                else:
                    loss_all=np.load(os.path.join(img_dir,str(batch_num)+loss_post));
                    if im_num>loss_all.shape[0]:
                        continue;
                    loss_curr=loss_all[im_num-1,0];
                    loss_str="{:10.4f}".format(loss_curr);
                if not batch_str:
                    files_curr=[os.path.join(img_dir,str(im_num)+file_post_curr) for file_post_curr in file_post];
                else:
                    files_curr=[os.path.join(img_dir,str(batch_num)+'_'+str(im_num)+file_post_curr) for file_post_curr in file_post];
                files_curr=[util.getRelPath(file_curr,dir_server) for file_curr in files_curr];
                captions_curr=[os.path.split(file_curr)[1]+' '+loss_str for file_curr in files_curr];
                if img_caption_pre is not None:
                    captions_curr=[img_caption_pre[idx_img_dir]+' '+caption_curr for caption_curr in captions_curr];
                files_curr_list.extend(files_curr);
                captions_curr_list.extend(captions_curr);
                
                img_files_all.append(files_curr_list);
                captions_all.append(captions_curr_list);

    visualize.writeHTML(out_file_html,img_files_all,captions_all,224,224);
    

def parseLogFile(log_file,gt_loss=False):
    log_data=util.readLinesFromFile(log_file);
    final_loss_str='minibatches processed: all'

    final_loss=[line_curr for line_curr in log_data if final_loss_str in line_curr];
    if len(final_loss)!=1:
        final_loss='None'
    else:
        final_loss=final_loss[0].split(' ');
        final_loss=final_loss[-1].strip('"');
    gt_str='gt val loss'
    
    
    if gt_loss:
        gt_losses=[line_curr for line_curr in log_data if gt_str in line_curr];
        gt_losses=[line_curr.split(' ')[-1].strip('"') for line_curr in gt_losses];
        gt_losses=' '.join(gt_losses);
    
    if gt_loss:
        return final_loss,gt_losses
    else:
        return final_loss;


def getCommandTestTrainImages(path_to_th,
                              outDir,
                              num_ctrl_pts,
                              val_horse_data_path,
                              val_human_data_path,
                             out_dir_images):
    command=['th',path_to_th];
    command.extend(['-outDir',outDir]);
    command.extend(['-model',os.path.join(outDir,'final','model_all_final.dat')])
    command.extend(['-num_ctrl_pts',str(num_ctrl_pts)]);
    command.extend(['-out_dir_images',out_dir_images]);
    command.extend(['-val_horse_data_path',val_horse_data_path]);
    command.extend(['-val_human_data_path',val_human_data_path]);
    command=' '.join(command);
    return command;

def getCommandLearningRateSearch(path_to_th,
                                outDir,
                                learningRate,
                                divisor,
                                multiplier):
    command=['th',path_to_th];
    command.extend(['-outDir',outDir]);
    command.extend(['learningRate',str(learningRate)]);
    command.extend(['divisor',str(divisor)]);
    command.extend(['multiplier',str(multiplier)]);
    command=' '.join(command);
    return command;

def comparePerformanceHTML(out_file_html,dirs_meta,dirs_str,gt_loss=False):
    test_dirs=[os.path.join(dir_curr,'test_images') for dir_curr in dirs_meta];
    log_files=[os.path.join(dir_curr,'log_test.txt') for dir_curr in test_dirs];
    loss_ims=[os.path.join(dir_curr,'intermediate','loss_all.png') for dir_curr in dirs_meta];
    
    losses=[];
    for file_curr in log_files:
        if os.path.exists(file_curr):
            losses.append(parseLogFile(file_curr,gt_loss));
        else:
            losses.append('No FILE');
        
#     losses=[parseLogFile(file_curr,gt_loss) for file_curr in log_files];
    
    # for loss_curr,dir_curr in zip(losses,dirs_meta):
    #     print loss_curr,dirs_meta;


    imgs=[];
    captions=[];
    for idx,file_curr in enumerate(loss_ims):
        if gt_loss:
            loss_curr=losses[idx][0]
        else:
            loss_curr=losses[idx];
        caption_curr=dirs_str[idx]+' '+loss_curr;
        captions.append([caption_curr]);
        imgs.append([util.getRelPath(file_curr,dir_server)]);

    visualize.writeHTML(out_file_html,imgs,captions,500,500);


dir_server='/home/SSD3/maheen-data';
click_str='http://vision1.idav.ucdavis.edu:1000'

In [31]:

dirs_meta=['/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_25_small_declr',
           '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_25_small_declr/epoch_4',
            '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_36_small_declr',
            '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_64_small_declr',
            '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_64_declr/quick_decrease',
              '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_64',
          '/home/SSD3/maheen-data/temp/tps_train_allKP_adam_noBad_bn'];
num_ctrl_pts=[25,25,36,64,64,64,36];
path_to_th='/home/maheenrashid/Downloads/horses/torch/train_tps_var_lr.th'
out_dir_images='train_images';
path_common='/home/SSD3/maheen-data/horse_project/data_check'

# val_horse_data_path=os.path.join(path_common,'horse/matches_5_val_allKP_minLoss_clean.txt');
# val_human_data_path=os.path.join(path_common,'aflw/matches_5_val_allKP_minLoss_noIm_clean.txt');

val_horse_data_path=os.path.join(path_common,'horse','matches_5_train_allKP_minLoss_clean_subset.txt')
val_human_data_path=os.path.join(path_common,'aflw','matches_5_train_allKP_minLoss_noIm_clean_subset.txt')

commands=[];
out_file_commands=os.path.join(dirs_meta[0],'test_on_train.sh');
for num_ctrl_pts,outDir in zip(num_ctrl_pts,dirs_meta):
    commands.append(getCommandTestTrainImages(path_to_th,outDir,num_ctrl_pts,\
                                              val_horse_data_path,val_human_data_path,out_dir_images));
print out_file_commands
util.writeFile(out_file_commands,commands);


    


/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_25_small_declr/test_on_train.sh


In [21]:


horse_data_path=os.path.join(path_common,'horse','matches_5_train_allKP_minLoss_clean.txt')
human_data_path=os.path.join(path_common,'aflw','matches_5_train_allKP_minLoss_clean.txt')
human_data_path_noIm=os.path.join(path_common,'aflw','matches_5_train_allKP_minLoss_noIm_clean.txt')

files=[horse_data_path,human_data_path,human_data_path_noIm];
out_files=[file_curr[:file_curr.rindex('.')]+'_subset.txt' for file_curr in files];

data_horse=util.readLinesFromFile(horse_data_path);
idx=range(len(data_horse));
random.shuffle(idx);
to_select=193;
idx=idx[:to_select];


for in_file,out_file in zip(files,out_files):
    data_curr=util.readLinesFromFile(in_file);
    data_out=[data_curr[idx_curr] for idx_curr in idx]
    print len(data_curr),len(data_out);
    print out_file
    util.writeFile(out_file,data_out);




3531 193
/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_train_allKP_minLoss_clean_subset.txt
3531 193
/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_train_allKP_minLoss_clean_subset.txt
3531 193
/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_train_allKP_minLoss_noIm_clean_subset.txt


In [33]:
dirs_meta=['/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_25_small_declr',
              '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_25_small_declr/epoch_4',
            '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_36_small_declr',
            '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_64_small_declr',
            '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_64_declr/quick_decrease',
              '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_64',
          '/home/SSD3/maheen-data/temp/tps_train_allKP_adam_noBad_bn'];

dirs_strs=['25 small dec','25 small dec diff','36 small dec','64 small dec','64 big dec','64 big nodec','orig 36 small nodec']

out_file_html=os.path.join(dirs_meta[0],'comparativeLosses.html');
dir_server='/home/SSD3/maheen-data';
click_str='http://vision1.idav.ucdavis.edu:1000'

comparePerformanceHTML(out_file_html,dirs_meta,dirs_str,True);
print out_file_html.replace(dir_server,click_str)


http://vision1.idav.ucdavis.edu:1000/horse_project/tps_bn_clean_var_lr_25_small_declr/comparativeLosses.html


In [41]:
dirs_meta=['/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_25_small_declr',
           '/home/SSD3/maheen-data/horse_project/tps_bn_clean_var_lr_36_small_declr',
          '/home/SSD3/maheen-data/temp/tps_train_allKP_adam_noBad_bn'];
img_dirs=[os.path.join(dir_curr,'test_images') for dir_curr in dirs_meta];
file_post=['_horse.jpg','_gtwarp.jpg','_predwarp.jpg',];
img_caption_pre=['25 small dec','36 small dec','orig 36 small nodec']
range_batches=[1,2];
range_images=range(1,101);
dir_server='/home/SSD3/maheen-data';
click_str='http://vision1.idav.ucdavis.edu:1000';
out_file_html=os.path.join(dirs_meta[0],'comparison_viz.html');
loss_post=None;
comparativeLossViz(img_dirs,file_post,loss_post,range_batches,range_images,
                   out_file_html,dir_server,img_caption_pre)
print out_file_html.replace(dir_server,click_str);



http://vision1.idav.ucdavis.edu:1000/horse_project/tps_bn_clean_var_lr_25_small_declr/comparison_viz.html


In [56]:
dirs_meta=['/home/SSD3/maheen-data/horse_project/ft_horse_allKp_full_big_clean_fullBack_varlr_25/start_1e-2',
           '/home/SSD3/maheen-data/horse_project/vanilla_ft_horse_1e-2/resume_1e-3'];

img_dirs=[os.path.join(dir_curr,'test_images') for dir_curr in dirs_meta];
file_post=['_org.jpg']
img_caption_pre=['warp ft 25','no warp ft']
range_batches=[1,2];
range_images=range(1,101);
dir_server='/home/SSD3/maheen-data';
click_str='http://vision1.idav.ucdavis.edu:1000';
out_file_html=os.path.join(dirs_meta[0],'comparison_viz.html');
loss_post=None;
comparativeLossViz(img_dirs,file_post,loss_post,range_batches,range_images,
                   out_file_html,dir_server,img_caption_pre)
print out_file_html.replace(dir_server,click_str);


http://vision1.idav.ucdavis.edu:1000/horse_project/ft_horse_allKp_full_big_clean_fullBack_varlr_25/start_1e-2/comparison_viz.html


In [15]:
dir_meta='/home/SSD3/maheen-data/temp/train_tps_full_system_lr_search';
path_to_th='/home/maheenrashid/Downloads/horses/torch/train_tps_full_system.th'
util.mkdir(dir_meta);
divisors=[1,10,100];
multipliers=[0,1,10,100];
learningRates=[1e-2,1e-3,1e-4];
num_files=1

commands=[];
for div in divisors:
    for mul in multipliers:
        for lr in learningRates:
            out_dir_curr=os.path.join(dir_meta,str(lr)+'_'+str(div)+'_'+str(mul))
            loss_check_file=os.path.join(out_dir_curr,'test_images','loss_final_val.npy');
            loss_log_file=os.path.join(out_dir_curr,'test_images','log_test.txt');
            loss_plot_file=os.path.join(out_dir_curr,'intermediate','loss_all.png');
            if not os.path.exists(loss_check_file) and os.path.exists(loss_log_file):
#             if not os.path.exists(loss_log_file):
                command_curr=getCommandLearningRateSearch(path_to_th,out_dir_curr,lr,div,mul);
                commands.append(command_curr);
            
random.shuffle(commands);
print len(commands);
out_files=[os.path.join(dir_meta,str(num)+'.sh') for num in range(4)];
print out_files;
# str_check=[str(num) for num in range(len(commands))]
num_one_file=len(commands)/num_files;
commands_broken=[commands[num_one_file*i:num_one_file*i+num_one_file] for i in range(num_files)];
for out_file,commands_curr in zip(out_files,commands_broken):
    print out_file,len(commands_curr)
    util.writeFile(out_file,commands_curr);




3
['/home/SSD3/maheen-data/temp/train_tps_full_system_lr_search/0.sh', '/home/SSD3/maheen-data/temp/train_tps_full_system_lr_search/1.sh', '/home/SSD3/maheen-data/temp/train_tps_full_system_lr_search/2.sh', '/home/SSD3/maheen-data/temp/train_tps_full_system_lr_search/3.sh']
/home/SSD3/maheen-data/temp/train_tps_full_system_lr_search/0.sh 3


In [30]:
dir_meta='/home/SSD3/maheen-data/temp/train_tps_kpFixed_system_lr_search';
util.mkdir(dir_meta)
path_to_th='/home/maheenrashid/Downloads/horses/torch/train_tps_full_system.th'
util.mkdir(dir_meta);
divisors=[0];
# [100,1000,0];
multipliers=[0];
learningRates=[1e-2,1e-3]
# [1e-4,1e-5,1e-6];
num_files=1

commands=[];
out_dirs=[];
for div in divisors:
    for mul in multipliers:
        for lr in learningRates:
            out_dir_curr=os.path.join(dir_meta,str(lr)+'_'+str(div)+'_'+str(mul))
            out_dirs.append(out_dir_curr);

#             loss_check_file=os.path.join(out_dir_curr,'test_images','loss_final_val.npy');
#             loss_log_file=os.path.join(out_dir_curr,'test_images','log_test.txt');
#             loss_plot_file=os.path.join(out_dir_curr,'intermediate','loss_all.png');
#             if not os.path.exists(loss_check_file) and os.path.exists(loss_log_file):
#             if not os.path.exists(loss_log_file):
            command_curr=getCommandLearningRateSearch(path_to_th,out_dir_curr,lr,div,mul);
            commands.append(command_curr);
            
random.shuffle(commands);
print len(commands);
out_files=[os.path.join(dir_meta,str(num)+'.sh') for num in range(4)];
# print out_files;
# str_check=[str(num) for num in range(len(commands))]
num_one_file=len(commands)/num_files;
commands_broken=[commands[num_one_file*i:num_one_file*i+num_one_file] for i in range(num_files)];
for out_file,commands_curr in zip(out_files,commands_broken):
    print out_file,len(commands_curr)
    util.writeFile(out_file,commands_curr);
    

out_file_html=os.path.join(dir_meta,'loss_comparison.html');

dirs_str=[os.path.split(dir_curr)[1] for dir_curr in out_dirs]

out_file_html=os.path.join(dir_meta,'comparativeLosses.html');
dir_server='/home/SSD3/maheen-data';
click_str='http://vision1.idav.ucdavis.edu:1000'

comparePerformanceHTML(out_file_html,out_dirs,dirs_str);
print out_file_html.replace(dir_server,click_str);


2
/home/SSD3/maheen-data/temp/train_tps_kpFixed_system_lr_search/0.sh 2
http://vision1.idav.ucdavis.edu:1000/temp/train_tps_kpFixed_system_lr_search/comparativeLosses.html


In [24]:
dir_meta='/home/SSD3/maheen-data/temp/train_tps_full_system_lr_search';
path_to_th='/home/maheenrashid/Downloads/horses/torch/train_tps_full_system.th'
util.mkdir(dir_meta);
divisors=[1,10,100];
multipliers=[0,1,10,100];
learningRates=[1e-2,1e-3,1e-4];

out_dirs=[]

for div in divisors:
    for mul in multipliers:
        for lr in learningRates:
            out_dir_curr=os.path.join(dir_meta,str(lr)+'_'+str(div)+'_'+str(mul))
            out_dirs.append(out_dir_curr);

out_file_html=os.path.join(dir_meta,'loss_comparison.html');

dirs_str=[os.path.split(dir_curr)[1] for dir_curr in out_dirs]

out_file_html=os.path.join(dir_meta,'comparativeLosses.html');
dir_server='/home/SSD3/maheen-data';
click_str='http://vision1.idav.ucdavis.edu:1000'

comparePerformanceHTML(out_file_html,out_dirs,dirs_str);
print out_file_html.replace(dir_server,click_str);


http://vision1.idav.ucdavis.edu:1000/temp/train_tps_full_system_lr_search/comparativeLosses.html


In [23]:
dir_curr='/home/SSD3/maheen-data/temp/train_tps_full_system_lr_search/0.001_100_1/test_images';
visualize.writeHTMLForFolder(dir_curr);
print os.path.join(dir_curr,os.path.split(dir_curr)[1]+'.html').replace(dir_server,click_str);

http://vision1.idav.ucdavis.edu:1000/temp/train_tps_full_system_lr_search/0.001_100_1/test_images/test_images.html


In [9]:
dir_meta='/home/SSD3/maheen-data/temp/analyze_tps';
pre_path=os.path.join(dir_meta,'tps_pre_128_50');
post_path=os.path.join(dir_meta,'tps_post_128_50_0_1e-2');
# pre_path=os.path.join(dir_meta,'vanilla_pre_horse');
# post_path=os.path.join(dir_meta,'vanilla_post_horse');
dirs=[pre_path,post_path];
out_file_html=os.path.join(dir_meta,'compare_fix.html');
img_paths=[];
captions=[];
for i in range(1,23):
    img_curr=str(i)+'.png';
    img_paths.append([util.getRelPath(os.path.join(dir_curr,img_curr),dir_server) for dir_curr in dirs]);
    captions.append(['pre','post']);

visualize.writeHTML(out_file_html,img_paths,captions,500,500);
print out_file_html.replace(dir_server,click_str);

http://vision1.idav.ucdavis.edu:1000/temp/analyze_tps/compare_fix.html


In [2]:
img_dirs=[os.path.join(dir_server,'horse_project/train_tps_full_system_1e-3_100_1/resume_1e-4/test_images')];
file_post=['.jpg','_org.jpg'];
out_file_html=os.path.join(img_dirs[0],'viz.html');
range_images=range(1,65);
range_batches=range(1,4);
loss_post=None;
comparativeLossViz(img_dirs,file_post,loss_post,range_batches,range_images,out_file_html,dir_server)
print out_file_html.replace(dir_server,click_str)

http://vision1.idav.ucdavis.edu:1000/horse_project/train_tps_full_system_1e-3_100_1/resume_1e-4/test_images/viz.html


In [8]:
img_dirs=[os.path.join\
          (dir_server,'horse_project/tps_bn_clean_var_lr_25_small_declr/test_images'),os.path.join\
          (dir_server,'horse_project/tps_25_1e-2_0/test_images')];

file_post=['_horse.jpg','_gtwarp.jpg','_predwarp.jpg'];
out_file_html=os.path.join(dir_server,'horse_project/tps_25_1e-2_0/comparison.html')
img_caption_pre=['finetune','fixed']
range_images=range(1,101);
range_batches=range(1,3);
loss_post=None;
comparativeLossViz(img_dirs,file_post,loss_post,range_batches,range_images,out_file_html,dir_server,img_caption_pre)
print out_file_html.replace(dir_server,click_str)

http://vision1.idav.ucdavis.edu:1000/horse_project/tps_25_1e-2_0/comparison.html
